In [4]:
import sys
import os
module_path = os.path.abspath('E:/Codes/inDox/libs/indoxArcg')
if module_path not in sys.path:
    sys.path.append(module_path)




In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

In [6]:
from indoxArcg.llms import OpenAi
from indoxArcg.embeddings import OpenAiEmbedding
from indoxArcg.data_loaders import Txt
from indoxArcg.splitter import RecursiveCharacterTextSplitter
from indoxArcg.pipelines.cag import CAG, KVCache

In [7]:
llm = OpenAi(model="gpt-4o-mini",api_key=OPENAI_API_KEY)
embed_model = OpenAiEmbedding(api_key=OPENAI_API_KEY, model="text-embedding-3-small")
txt_loader = Txt(txt_path="sample.txt")
splitter = RecursiveCharacterTextSplitter()

INFO: Initializing OpenAi with model: gpt-4o-mini
INFO: OpenAi initialized successfully
INFO: Initialized OpenAiEmbedding with model: text-embedding-3-small


In [8]:
docs = txt_loader.load()
split_docs = splitter.split_text(text=docs)
print(len(split_docs))
print(split_docs[0])


38
The wife of a rich man fell sick, and as she felt that her end

was drawing near, she called her only daughter to her bedside and

said, dear child, be good and pious, and then the

good God will always protect you, and I will look down on you

from heaven and be near you.  Thereupon she closed her eyes and

departed.  Every day the maiden went out to her mother's grave,


In [9]:
cag_with_embedding = CAG(
    llm=llm,
    
    embedding_model=embed_model,
    cache=KVCache(),
)
cache_embed_key = "embed_cache"
cag_with_embedding.preload_documents(split_docs,cache_embed_key)




cache_no_embed_key = "no_embed_cache"
cag_without_embedding = CAG(llm=llm, cache=KVCache())
cag_without_embedding.preload_documents(split_docs, cache_no_embed_key)


INFO: Precomputing KV cache for 38 document chunks...


2025-01-21 09:30:30,846 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-01-21 09:30:32,442 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-01-21 09:30:33,333 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-01-21 09:30:34,363 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-01-21 09:30:35,015 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-01-21 09:30:36,933 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-01-21 09:30:38,196 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-01-21 09:30:39,326 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-01-21 09:30:40,692 - httpx - INFO - HTTP Request: POST https://api.openai.c

INFO: KV cache saved: kv_cache\embed_cache.pkl
INFO: Preloaded 38 document chunks into KV cache
INFO: Precomputing KV cache for 38 document chunks...
INFO: KV cache saved: kv_cache\no_embed_cache.pkl
INFO: Preloaded 38 document chunks into KV cache


In [10]:
query = "How Cinderella reach her happy ending?"

In [12]:
from pprint import pprint
pprint(cag_with_embedding.infer(query,cache_key=cache_embed_key))

INFO: Retrieving relevant context...


2025-01-21 09:31:45,693 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


INFO: Selected 5 relevant chunks from cache
INFO: Performing inference with filtered context...


2025-01-21 09:31:49,282 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


('Cinderella reached her happy ending through a series of magical events and '
 'her unwavering kindness. After being mistreated by her stepmother and '
 'stepsisters, she received help from a magical bird that granted her a '
 'beautiful dress and slippers, allowing her to attend the royal wedding. At '
 'the event, the prince recognized her as the beautiful maiden he had danced '
 "with before. Despite her stepfamily's attempts to conceal her identity, the "
 'prince took Cinderella away on his horse, leading to their eventual '
 'happiness together. The support of the magical tree and the bird, along with '
 'her resilience and goodness, ultimately led to her transformation from a '
 'mistreated girl to a beloved princess.')


In [13]:
pprint(cag_without_embedding.infer(query,cache_key=cache_no_embed_key))

INFO: Loading KV cache for key: no_embed_cache
INFO: KV cache loaded: kv_cache\no_embed_cache.pkl
INFO: Retrieving relevant context...
INFO: Selected 0 relevant chunks from cache
INFO: Performing inference with filtered context...


2025-01-21 09:32:16,351 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


('Cinderella reaches her happy ending through a series of fortunate events '
 'that begin with her kindness and resilience despite her difficult '
 'circumstances. After being mistreated by her stepmother and stepsisters, she '
 'receives help from her Fairy Godmother, who magically transforms her '
 'appearance and provides her with a beautiful gown and glass slippers to '
 'attend the royal ball. At the ball, she captures the heart of the prince, '
 'but she must leave before midnight, when the magic wears off. In her haste, '
 'she leaves behind one of her glass slippers. The prince searches the kingdom '
 'for the owner of the slipper, and when he finds Cinderella, he realizes she '
 "is the one he loves. They are reunited, and Cinderella's kindness and true "
 'identity are recognized, leading to her marriage with the prince and a life '
 'filled with happiness.')
